In [19]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# --- 1. Constants and Configuration ---

# This should match the MAX_SEQ_LEN from the Transformer model for consistency.
MAX_SEQ_LEN = 128

# --- 2. Data Loading and Preprocessing ---

def load_and_preprocess_data(proportions_file, ground_truth_file):
    """
    Loads predicted proportions, extracts statistical features, and aligns them with
    the true total time for each sequence.
    """
    if not os.path.exists(proportions_file):
        print(f"❌ Error: Proportions file not found at '{proportions_file}'")
        return None, None, None, None, None
    if not os.path.exists(ground_truth_file):
        print(f"❌ Error: Ground truth data file not found at '{ground_truth_file}'")
        return None, None, None, None, None

    props_df = pd.read_csv(proportions_file)
    truth_df = pd.read_csv(ground_truth_file)
    
    # --- Re-calculate the true total time using the definitive logic ---
    truth_df['step_duration'] = truth_df.groupby('SeqOrder')['timediff'].diff().fillna(truth_df['timediff'])
    truth_df['step_duration'] = truth_df['step_duration'].clip(lower=0)
    truth_df['Step'] = truth_df.groupby('SeqOrder').cumcount()
    
    end_marker_step = truth_df[truth_df['sourceID'] == 10].groupby('SeqOrder')['Step'].first()
    truth_df['end_marker_step'] = truth_df['SeqOrder'].map(end_marker_step)
    truth_df.loc[truth_df['Step'] > truth_df['end_marker_step'], 'step_duration'] = 0
    
    total_times = truth_df.groupby('SeqOrder')['step_duration'].sum()

    # --- Prepare data for the Models ---
    X_sequences, X_num_steps, X_stats = [], [], []
    
    for _, g in props_df.groupby('SeqOrder'):
        proportions = g['predicted_proportion'].values
        X_sequences.append(proportions.reshape(-1, 1))
        X_num_steps.append(len(g))
        
        # --- Feature Engineering: Create a richer set of statistical features ---
        stats = [
            np.mean(proportions), np.std(proportions), np.max(proportions),
            np.percentile(proportions, 25), np.median(proportions), np.percentile(proportions, 75)
        ]
        X_stats.append(stats)
    
    y_total_times = props_df['SeqOrder'].unique()
    y_sequences = np.array([total_times.get(seq_id, 0) for seq_id in y_total_times])

    X_padded_seq = tf.keras.preprocessing.sequence.pad_sequences(
        X_sequences, maxlen=MAX_SEQ_LEN, padding='post', dtype='float32'
    )
    
    X_steps_arr = np.array(X_num_steps, dtype='float32').reshape(-1, 1)
    X_stats_arr = np.array(X_stats, dtype='float32')

    print(f"Successfully processed {len(X_padded_seq)} sequences.")
    
    return X_padded_seq, X_steps_arr, X_stats_arr, y_sequences.reshape(-1, 1), props_df


# --- 3. Model Architectures ---

def build_statistical_model(scalar_shape, stats_shape):
    """Builds a simple MLP model based on high-level statistical features."""
    scalar_input = layers.Input(shape=scalar_shape, name='scalar_input')
    stats_input = layers.Input(shape=stats_shape, name='stats_input')
    
    concatenated = layers.concatenate([scalar_input, stats_input])
    x = layers.Dense(64, activation='relu')(concatenated)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, name='stat_output')(x)
    
    model = tf.keras.Model(inputs=[scalar_input, stats_input], outputs=outputs, name="StatisticalModel")
    return model

def build_sequential_model(sequence_shape):
    """Builds a powerful sequential model that ONLY sees the proportions."""
    sequence_input = layers.Input(shape=sequence_shape, name='sequence_input')
    
    masked_sequence = layers.Masking(mask_value=0.)(sequence_input)
    gru_out = layers.Bidirectional(layers.GRU(128, return_sequences=True))(masked_sequence)
    attention_out = layers.MultiHeadAttention(num_heads=8, key_dim=256)(query=gru_out, value=gru_out, key=gru_out)
    context_vector = layers.GlobalAveragePooling1D()(attention_out)
    
    x = layers.Dense(128, activation='relu')(context_vector)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1, name='seq_output')(x)
    
    model = tf.keras.Model(inputs=sequence_input, outputs=outputs, name="SequentialModel")
    return model

def build_meta_model(n_features):
    """Builds the meta-learner model."""
    meta_input = layers.Input(shape=(n_features,), name='meta_input')
    x = layers.Dense(32, activation='relu')(meta_input)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation='relu')(x)
    outputs = layers.Dense(1, name='meta_output')(x)
    
    model = tf.keras.Model(inputs=meta_input, outputs=outputs, name="MetaModel")
    return model

# --- 4. Visualization Function ---

def create_visualizations(results_df, output_dir='visualizations'):
    """Generates and saves plots comparing true vs. predicted total time."""
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    true_times, predicted_times = results_df['true_total_time'], results_df['predicted_total_time']
    plt.figure(figsize=(10, 10))
    plt.scatter(true_times, predicted_times, alpha=0.6)
    lims = [np.min([plt.xlim(), plt.ylim()]), np.max([plt.xlim(), plt.ylim()])]
    plt.plot(lims, lims, 'r--', alpha=0.75, zorder=0)
    plt.xlabel("True Total Time (seconds)"), plt.ylabel("Predicted Total Time (seconds)"), plt.title("True vs. Predicted Total Time"), plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'true_vs_predicted_scatter.png')), plt.close()
    errors = predicted_times - true_times
    plt.figure(figsize=(10, 6))
    plt.hist(errors, bins=30, alpha=0.7)
    plt.xlabel("Prediction Error (Predicted - True)"), plt.ylabel("Frequency"), plt.title("Distribution of Prediction Errors"), plt.grid(True), plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
    plt.savefig(os.path.join(output_dir, 'prediction_error_histogram.png')), plt.close()
    print(f"✅ Basic visualizations saved to '{output_dir}' directory.")

def create_advanced_visualizations(results_df, output_dir='visualizations'):
    """Generates and saves advanced diagnostic plots."""
    true_times, predicted_times = results_df['true_total_time'], results_df['predicted_total_time']
    residuals = true_times - predicted_times
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=predicted_times, y=residuals, alpha=0.6)
    plt.axhline(y=0, color='r', linestyle='--'), plt.xlabel("Predicted Total Time (seconds)"), plt.ylabel("Residuals (True - Predicted)"), plt.title("Residuals vs. Predicted Values"), plt.grid(True)
    plt.savefig(os.path.join(output_dir, 'residuals_vs_predicted.png')), plt.close()
    plt.figure(figsize=(10, 6))
    sns.histplot(true_times, color="blue", label='True Values', kde=True, stat="density", linewidth=0)
    sns.histplot(predicted_times, color="red", label='Predicted Values', kde=True, stat="density", linewidth=0)
    plt.title("Distribution of Predicted vs. True Values"), plt.xlabel("Total Time (seconds)"), plt.legend()
    plt.savefig(os.path.join(output_dir, 'predicted_vs_true_distribution.png')), plt.close()
    print(f"✅ Advanced visualizations saved to '{output_dir}' directory.")


# --- 5. Training and Prediction Orchestration ---

def main():
    """Main function to run the data processing, training, and prediction."""
    
    proportions_file = 'prediction_176401_proportions_final_all.csv'
    ground_truth_file = 'data/176401/encoded_176401_condensed_full.csv'
    output_predictions_file = 'prediction_176401_total_time_full.csv'
    
    X_seq, X_steps, X_stats, y, props_df = load_and_preprocess_data(proportions_file, ground_truth_file)
    if X_seq is None: return

    X_seq_train, X_seq_val, X_steps_train, X_steps_val, X_stats_train, X_stats_val, y_train, y_val = train_test_split(
        X_seq, X_steps, X_stats, y, test_size=0.2, random_state=42
    )
    
    scaler_steps = StandardScaler(); X_steps_train_scaled, X_steps_val_scaled = scaler_steps.fit_transform(X_steps_train), scaler_steps.transform(X_steps_val)
    scaler_stats = StandardScaler(); X_stats_train_scaled, X_stats_val_scaled = scaler_stats.fit_transform(X_stats_train), scaler_stats.transform(X_stats_val)
    scaler_y = StandardScaler(); y_train_scaled, y_val_scaled = scaler_y.fit_transform(y_train), scaler_y.transform(y_val)
    
    # --- Train Statistical Model ---
    print("\n--- Training Statistical Model ---")
    stat_model = build_statistical_model((1,), X_stats_train.shape[1:])
    stat_model.compile(optimizer='adam', loss='huber', metrics=['mae'])
    stat_model.fit(
        [X_steps_train_scaled, X_stats_train_scaled], y_train_scaled,
        validation_data=([X_steps_val_scaled, X_stats_val_scaled], y_val_scaled),
        epochs=300, batch_size=32,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)],
        verbose=0
    )
    
    # --- Train Sequential Model ---
    print("\n--- Training Sequential Model ---")
    seq_model = build_sequential_model(X_seq_train.shape[1:])
    seq_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='huber', metrics=['mae'])
    seq_model.fit(
        X_seq_train, y_train_scaled,
        validation_data=(X_seq_val, y_val_scaled),
        epochs=500, batch_size=32,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=75, restore_best_weights=True)],
        verbose=0
    )
    
    # --- Create Training Data for Meta-Model ---
    stat_preds_val = stat_model.predict([X_steps_val_scaled, X_stats_val_scaled])
    seq_preds_val = seq_model.predict(X_seq_val)
    meta_X_train = np.hstack((stat_preds_val, seq_preds_val, X_steps_val_scaled, X_stats_val_scaled))
    meta_y_train = y_val_scaled
    
    # --- Train Meta-Model ---
    print("\n--- Training Meta-Model ---")
    meta_model = build_meta_model(meta_X_train.shape[1])
    meta_model.compile(optimizer='adam', loss='huber', metrics=['mae'])
    meta_model.summary()
    meta_model.fit(
        meta_X_train, meta_y_train,
        epochs=200, batch_size=16,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)],
        verbose=1
    )
    
    # --- Generate Final Predictions ---
    print("\n--- Generating Final Ensemble Predictions ---")
    X_steps_scaled, X_stats_scaled = scaler_steps.transform(X_steps), scaler_stats.transform(X_stats)
    stat_preds_scaled = stat_model.predict([X_steps_scaled, X_stats_scaled])
    seq_preds_scaled = seq_model.predict(X_seq)
    
    meta_X_final = np.hstack((stat_preds_scaled, seq_preds_scaled, X_steps_scaled, X_stats_scaled))
    final_preds_scaled = meta_model.predict(meta_X_final)
    
    predicted_times = scaler_y.inverse_transform(final_preds_scaled).flatten()
    
    # --- Final Output ---
    seq_order_to_time = dict(zip(props_df['SeqOrder'].unique(), predicted_times))
    props_df['predicted_total_time'] = np.nan
    end_marker_indices = props_df[props_df['sourceID'] == 10].groupby('SeqOrder')['Step'].idxmin()
    for seq_order, idx in end_marker_indices.items():
        if seq_order in seq_order_to_time:
            props_df.loc[idx, 'predicted_total_time'] = seq_order_to_time[seq_order]

    props_df.to_csv(output_predictions_file, index=False)
    print(f"✅ Final predictions saved to '{output_predictions_file}'")

    results_df = pd.DataFrame({
        'SeqOrder': props_df['SeqOrder'].unique(),
        'true_total_time': scaler_y.inverse_transform(y).flatten(),
        'predicted_total_time': predicted_times
    })
    
    create_visualizations(results_df)
    create_advanced_visualizations(results_df)

    print("\n--- Sample of Final Predictions ---")
    if not props_df.empty: print(props_df[props_df['SeqOrder'] == props_df['SeqOrder'].iloc[0]])


In [21]:
if __name__ == "__main__":
    main()

Successfully processed 223 sequences.

--- Training Statistical Model ---


C:\Users\lukis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['scalar_input', 'stats_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(



--- Training Sequential Model ---


C:\Users\lukis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\lukis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\lukis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\layer.py

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 495ms/step

--- Training Meta-Model ---


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ meta_input (InputLayer)         │ (None, 9)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 32)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ meta_output (Dense)             │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.1790 - mae: 0.4035
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2180 - mae: 0.4461 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1786 - mae: 0.3985 7
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1331 - mae: 0.3426
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1822 - mae: 0.3764  
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2619 - mae: 0.4636
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2713 - mae: 0.4877
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3212 - mae: 0.5218
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3154 - mae: 0.4968
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1685 - mae: 0.3379  
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1787 - mae: 0.3313
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2890 - mae: 0.4936
Epoch 13/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - l